In [1]:
import pandas as pd

from risk_icu.preprocessing import preprocessing_24_hour

from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import roc_auc_score

from imblearn.over_sampling import SMOTE

## Data

In [2]:
data = pd.read_csv("raw_data/training_v2.csv")
X, y = preprocessing_24_hour(data)

/Users/franciscochaves/code/DomNau1001/Risk_ICU/risk_icu/preprocessing.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_post_cats[ohe.get_feature_names_out()] = ohe.transform(X_post_cats[["ethnicity"]])
/Users/franciscochaves/code/DomNau1001/Risk_ICU/risk_icu/preprocessing.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_post_cats[ohe.get_feature_names_out()] = ohe.transform(X_post_cats[["ethnicity"]])
/Users/franciscochaves/code/DomNau1001/Risk_ICU/risk_icu/preprocessing.py:69: SettingWithCopyW

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, stratify=y)

In [4]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train)

## Data Balancing

In [5]:
X_res, y_res = SMOTE().fit_resample(X_train, y_train)

## SVC

In [6]:
svc = SVC(kernel = "sigmoid", C = 9.18, coef0=0.14, gamma = "scale", probability = True, class_weight="balanced")

In [7]:
svc.fit(X_res, y_res)

/Users/franciscochaves/.pyenv/versions/3.10.6/envs/Risk_ICU/lib/python3.10/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=9.18, class_weight='balanced', coef0=0.14, kernel='sigmoid',
    probability=True)

In [8]:
cross_validate(svc, X_test, y_test, cv = 5, scoring = "recall")["test_score"].mean()

/Users/franciscochaves/.pyenv/versions/3.10.6/envs/Risk_ICU/lib/python3.10/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/franciscochaves/.pyenv/versions/3.10.6/envs/Risk_ICU/lib/python3.10/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/franciscochaves/.pyenv/versions/3.10.6/envs/Risk_ICU/lib/python3.10/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/franciscochaves/.pyenv/versions/3.10.6/envs/Risk_ICU/lib/

0.6167522025489693

In [9]:
y_score = svc.predict_proba(X_test)[:, 1]
y_true = y_test
auc_score = roc_auc_score(y_true, y_score)
print("ROC AUC Score:", auc_score)

ROC AUC Score: 0.6557985256843626


## Naive Bayes

In [10]:
nb = GaussianNB(var_smoothing=5.964585534596374e-06)

nb.fit(X_res, y_res)

/Users/franciscochaves/.pyenv/versions/3.10.6/envs/Risk_ICU/lib/python3.10/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GaussianNB(var_smoothing=5.964585534596374e-06)

In [11]:
from sklearn.metrics import recall_score
y_pred = nb.predict(X_test)
cross_validate(nb, X_test, y_test, cv = 5, scoring = "recall")["test_score"].mean(), recall_score(y_test, y_pred)

/Users/franciscochaves/.pyenv/versions/3.10.6/envs/Risk_ICU/lib/python3.10/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/franciscochaves/.pyenv/versions/3.10.6/envs/Risk_ICU/lib/python3.10/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/franciscochaves/.pyenv/versions/3.10.6/envs/Risk_ICU/lib/python3.10/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/franciscochaves/.pyenv/versions/3.10.6/envs/Risk_ICU/lib/

(0.5363057052433496, 0.6532593619972261)

In [12]:
y_score = nb.predict_proba(X_test)[:, 1]
y_true = y_test
auc_score = roc_auc_score(y_true, y_score)
print("ROC AUC Score:", auc_score)

ROC AUC Score: 0.8028149531754938


## Boosting and Bagging on NB

In [13]:
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [19]:
# AdaBoost
from sklearn.model_selection import cross_val_score

ada_boost_model = AdaBoostClassifier(nb, n_estimators=50, random_state=42)
ada_boost_model.fit(X_res, y_res)
ada_boost_pred = ada_boost_model.predict(X_test)
ada_boost_recall = recall_score(y_test, ada_boost_pred)
cross_val_score(ada_boost_model, X_test, y_test, cv = 5, scoring = "recall").mean(), ada_boost_recall

/Users/franciscochaves/.pyenv/versions/3.10.6/envs/Risk_ICU/lib/python3.10/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/franciscochaves/.pyenv/versions/3.10.6/envs/Risk_ICU/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/Users/franciscochaves/.pyenv/versions/3.10.6/envs/Risk_ICU/lib/python3.10/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/franciscochaves/.pyenv/versions/3.10.6/envs/Risk_ICU/lib/python3.10/site-pac

(0.6033775981524249, 0.9343504392048081)

In [25]:
from sklearn.model_selection import RandomizedSearchCV

param_dist = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.1, 0.5, 1.0],
    'algorithm': ['SAMME', 'SAMME.R'],
    'random_state': [42]
}

boosting_random_search = RandomizedSearchCV(
    estimator=AdaBoostClassifier(nb, random_state=42),
    param_distributions=param_dist,
    n_iter=10,
    cv=5,
    random_state=42
)
boosting_random_search.fit(X_res, y_res)

best_boosting_params = boosting_random_search.best_params_
best_boosting_model = AdaBoostClassifier(**best_boosting_params)
best_boosting_model.fit(X_res, y_res)

ada_boost_pred = best_boosting_model.predict(X_test)
ada_boost_recall = recall_score(y_test, ada_boost_pred)
cross_val_score(best_boosting_model, X_test, y_test, cv = 5, scoring = "recall").mean(), ada_boost_recall

/Users/franciscochaves/.pyenv/versions/3.10.6/envs/Risk_ICU/lib/python3.10/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/franciscochaves/.pyenv/versions/3.10.6/envs/Risk_ICU/lib/python3.10/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/franciscochaves/.pyenv/versions/3.10.6/envs/Risk_ICU/lib/python3.10/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/franciscochaves/.pyenv/versions/3.10.6/envs/Risk_ICU/lib/

(0.03051599520999059, 0.7027276930189551)

In [21]:
bagging_model = BaggingClassifier(nb, n_estimators=50, random_state=42)
bagging_model.fit(X_res, y_res)
bagging_pred = bagging_model.predict(X_test)
bagging_recall = recall_score(y_test, bagging_pred)
cross_val_score(bagging_model, X_test, y_test, cv = 5, scoring = "recall").mean(), bagging_recall


/Users/franciscochaves/.pyenv/versions/3.10.6/envs/Risk_ICU/lib/python3.10/site-packages/sklearn/ensemble/_bagging.py:782: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/franciscochaves/.pyenv/versions/3.10.6/envs/Risk_ICU/lib/python3.10/site-packages/sklearn/ensemble/_bagging.py:782: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/franciscochaves/.pyenv/versions/3.10.6/envs/Risk_ICU/lib/python3.10/site-packages/sklearn/ensemble/_bagging.py:782: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/franciscochaves/.pyenv/versions/3.10.6/envs/Risk_ICU/lib/

(0.5363067744418785, 0.6537216828478964)

In [23]:
param_dist = {
    'n_estimators': [50, 100, 200],
    'max_samples': [0.5, 0.7, 1.0],
    'max_features': [0.5, 0.7, 1.0],
    'bootstrap': [True, False],
    'bootstrap_features': [True, False],
    'random_state': [42]
}

bagging_random_search = RandomizedSearchCV(
    estimator=BaggingClassifier(nb, random_state=42),
    param_distributions=param_dist,
    n_iter=10,
    cv=5,
    random_state=42
)
bagging_random_search.fit(X_res, y_res)

best_bagging_params = bagging_random_search.best_params_
best_bagging_model = BaggingClassifier(**best_bagging_params)
best_bagging_model.fit(X_res, y_res)
best_bagging_pred = best_bagging_model.predict(X_test)
bagging_recall = recall_score(y_test, best_bagging_pred)
cross_val_score(best_bagging_model, X_test, y_test, cv = 5, scoring = "recall").mean(), bagging_recall


/Users/franciscochaves/.pyenv/versions/3.10.6/envs/Risk_ICU/lib/python3.10/site-packages/sklearn/ensemble/_bagging.py:782: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/franciscochaves/.pyenv/versions/3.10.6/envs/Risk_ICU/lib/python3.10/site-packages/sklearn/ensemble/_bagging.py:782: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/franciscochaves/.pyenv/versions/3.10.6/envs/Risk_ICU/lib/python3.10/site-packages/sklearn/ensemble/_bagging.py:782: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/franciscochaves/.pyenv/versions/3.10.6/envs/Risk_ICU/lib/

(0.1303695150115473, 0.22422561257512713)

## Decision Tree + Adaptive Boosting

In [ ]:
tree = DecisionTreeClassifier(class_weight = "balanced", min_samples_split=12, min_samples_leaf=6, max_leaf_nodes=45, max_features=None, max_depth = 2, criterion = "entropy", splitter = "random", min_impurity_decrease=0.0)
tree.fit(X_res, y_res)

ada_tree = AdaBoostClassifier(tree, n_estimators=100, learning_rate = 0.2, algorithm = "SAMME.R")
ada_tree.fit(X_res, y_res)


/Users/franciscochaves/.pyenv/versions/3.10.6/envs/Risk_ICU/lib/python3.10/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/franciscochaves/.pyenv/versions/3.10.6/envs/Risk_ICU/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoostClassifier(estimator=DecisionTreeClassifier(class_weight='balanced',
                                                    criterion='entropy',
                                                    max_depth=2,
                                                    max_leaf_nodes=45,
                                                    min_samples_leaf=6,
                                                    min_samples_split=12,
                                                    splitter='random'),
                   learning_rate=0.2, n_estimators=100)

In [ ]:
cross_validate(ada_tree, X_test, y_test, cv = 5, scoring = "recall")["test_score"].mean()

/Users/franciscochaves/.pyenv/versions/3.10.6/envs/Risk_ICU/lib/python3.10/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/franciscochaves/.pyenv/versions/3.10.6/envs/Risk_ICU/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/Users/franciscochaves/.pyenv/versions/3.10.6/envs/Risk_ICU/lib/python3.10/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/franciscochaves/.pyenv/versions/3.10.6/envs/Risk_ICU/lib/python3.10/site-pac

0.7202955264733556

In [ ]:
y_score = ada_tree.predict_proba(X_test)[:, 1]
y_true = y_test
auc_score = roc_auc_score(y_true, y_score)
print("ROC AUC Score:", auc_score)

ROC AUC Score: 0.8187403034590428


## Ensemble Model

In [ ]:
#1. -> BaggingClassifier and AdaBoostClassifier vor SVC and Naive Bayes
#2. -> Voting Classifier & Stacking Classifier combining all 3 models